### Objective
This notebook the raw `.dcd` files (along with some other files) to unwrap and concat the trajectories for visualization. Note that prior to running this Notebook,
the `scripts/00_unwrap_trajectory/read_raw_dcd.tcl` and `scripts/00_unwrap_trajectory/reload.tcl` files must be run as they generate the required input files (written to `scripts/00_unwrap_trajectory/output`).

In [60]:
import MDAnalysis as mda
import MDAnalysis.transformations.nojump
import MDAnalysis
import os
import MDAnalysis.transformations as trans

In [61]:
dcd_arr = []
SYSTEM="8a"
copynum="01"
dcd_dir = f"output/wnt{SYSTEM}"
dcd_list = sorted(os.listdir(f"{dcd_dir}"))[1:-1]

dcd_arr.append([f"{dcd_dir}/{name}" for name in dcd_list])
dcd_arr = [x for xs in dcd_arr for x in xs]
print(dcd_arr)

In [62]:
import MDAnalysis.transformations as trans
u = mda.Universe(f"output/wnt{SYSTEM}/total_{copynum}.psf", dcd_arr)
protein = u.select_atoms("all")

transforms = [MDAnalysis.transformations.nojump.NoJump(),
              trans.center_in_box(protein)]

u.trajectory.add_transformations(*transforms)


with MDAnalysis.Writer(f"output/wnt{SYSTEM}/total_{copynum}.dcd", protein.n_atoms) as W:
    for ts in u.trajectory:
        W.write(protein)

In [30]:
import MDAnalysis.analysis.rms
SYSTEM="4"
copy="01"
u = mda.Universe(f"output/wnt{SYSTEM}/total_{copy}.psf", f"output/wnt{SYSTEM}/total_{copy}.dcd")
ref = mda.Universe(f"output/wnt{SYSTEM}/total_{copy}.psf", f"output/wnt{SYSTEM}/Wnt4WlsPc_copy_02.pdb")  # open AdK (PDB ID: 4AKE)
R1 = MDAnalysis.analysis.rms.RMSD(u.select_atoms("name CA"), ref.select_atoms("name CA"), center=True)
R1.run()

SYSTEM="3a"
u = mda.Universe(f"output/wnt{SYSTEM}/total.psf", f"output/wnt{SYSTEM}/total.dcd")
protein = u.select_atoms("name CA")
R2 = MDAnalysis.analysis.rms.RMSD(u.select_atoms("name CA"), u.select_atoms("name CA"), center=True)
R2.run()

In [32]:
import matplotlib.pyplot as plt
rmsd1 = R1.rmsd.T   # transpose makes it easier for plotting
rmsd2 = R2.rmsd.T   # transpose makes it easier for plotting
time1 = rmsd1[1]*20
time2 = rmsd2[1]*20

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(time1, rmsd1[2], 'k-',  label="Wnt4a_copy1")
ax.plot(time2, rmsd2[2], 'b-',  label="Wnt3a_copy1")
ax.legend(loc="best")
ax.set_xlabel("time (ps)")
ax.set_ylabel(r"RMSD ($\AA$)")
fig.savefig("rmsd_all_CORE_LID_NMP_ref1AKE.pdf")

In [59]:
import numpy as np
names = ["1a", "3a", "4a", "5a","8a"]

for n in names:
    ref = mda.Universe(f"../00_map/input/Wnt{n}_align.pdb")
    numbers = ref.select_atoms("segid PROA").residues.resnums
    for num in range(len(numbers) - 1):
        if int(numbers[num] + 1) != int(numbers[num+1]):
            print(f"{n}: {num} - {len(numbers) - 1}")